In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import pandas as pd
import math

In [19]:
import pickle
import numpy as np
import pandas as pd
from geographiclib.geodesic import Geodesic
import os

In [167]:
# def get_all_filenames(folder_path):
#     filenames = []
#     for filename in os.listdir(folder_path):
#         filenames.append(filename)
#     return filenames

In [168]:
# folder_path = "D:\Spatial_interpolation\SSIN\data\india"
# all_filenames = get_all_filenames(folder_path)

In [169]:
# file_result=[]
# for filename in all_filenames:
#     x=pd.read_csv(folder_path+"/"+filename)
#     x=np.array(x)
#     x=np.reshape(x,(105120, 4 ,1))
#     file_result.append(x)

In [170]:
# result=file_result[0]

In [171]:
# for i in range(1,len(file_result)):
#     result = np.append(result,file_result[i] , axis=2)

In [39]:
X_raw=np.load('data/all.npy')

In [40]:
j=5
train_data=X_raw[:,j,:]
columns_with_missing_data = np.any(np.isnan(train_data), axis=0)
missing_columns = np.where(columns_with_missing_data)[0]
result = np.delete(train_data, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))

In [41]:
missing_columns

array([ 0, 34, 38, 49, 50, 55, 56, 73, 81, 83], dtype=int64)

In [42]:
result.shape

(8784, 93)

In [43]:
train_list=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61]
test_list=[62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92]
# train_list=[0,1,2,3]
# test_list=[4,5,6,7]

In [44]:
train_data=result[:,train_list]
test_data=result[:,test_list]

In [45]:
len(train_data[0])

62

In [46]:
train_data.shape

(8784, 62)

In [47]:
train_data = np.expand_dims(train_data, axis=-1)
test_data = np.expand_dims(test_data, axis=-1)

In [37]:
# train_data = np.transpose(train_data, (0, 2, 1))
# test_data = np.transpose(test_data, (0, 2, 1))

In [179]:
# train_data = np.reshape(train_data, (105120, 4,1))
# test_data = np.reshape(test_data, (105120, 4,1))

In [48]:
train_data.shape

(8784, 62, 1)

In [180]:
# delet_list=[]
# for i in range(len(train_data)):
#     temp=np.reshape(train_data[i],(4))
#     if (len(set(temp)) == 1):
#         delet_list.append(i)


In [181]:
# new_array=np.delete(train_data,delet_list,axis=0)

In [49]:
# test_data=np.delete(test_data,delet_list,axis=0)

NameError: name 'delet_list' is not defined

In [183]:
# test_data.shape

(104823, 4, 1)

In [50]:
stations = pd.read_csv("D:\Spatial_interpolation\SSIN\data\Station_info.csv")

In [51]:
df_train=stations.iloc[0:62]
df_test=stations.iloc[62:93]

In [93]:
# train_info=np.c_[df_train["lat"],df_train["lon"]]
# test_info=np.c_[df_test["lat"],df_test["lon"]]

In [96]:
# xls = np.linspace(0,100,100)
# yls = np.linspace(0,100,100)
# xgrid,ygrid = np.meshgrid(xls,yls)
# OK = OrdinaryKriging(
#     data[:, 0],
#     data[:, 1],
#     data[:, 2],
#     variogram_model="linear",
#     verbose=False,
#     enable_plotting=False,
# )

In [52]:
result=[]
for i in range(len(train_data)):
    # if i==4539:
    #     continue
    OK = OrdinaryKriging(
    df_train["lat"],
    df_train["lon"],
    train_data[i],
    variogram_model="linear",
    verbose=False,
    enable_plotting=False,
    )
    z, ss = OK.execute("points", df_test["lat"], df_test["lon"])
    result.append(z)

In [54]:
test_data = np.reshape(test_data, (272304))

In [55]:
result=np.array(result)

In [57]:
result=np.reshape(result, (272304 ))

## MAE

In [58]:
sum(abs(test_data-result))/(272304 )

0.247677700569703

## RMSE

In [59]:
import math
math.sqrt(sum((test_data-result)**2)/(272304 ))

0.29415856151021236

#### 其他的参数

In [60]:
result=[]
for i in range(len(train_data)):
    # if i==4539:
    #     continue
    OK = OrdinaryKriging(
    df_train["lat"],
    df_train["lon"],
    train_data[i],
    variogram_model="gaussian",
    verbose=False,
    enable_plotting=False,
    )
    z, ss = OK.execute("points", df_test["lat"], df_test["lon"])
    result.append(z)

In [61]:
result=np.array(result)
result=np.reshape(result, (272304))

In [62]:
sum(abs(test_data-result))/(272304)

86.77838655711672

In [63]:
math.sqrt(sum((test_data-result)**2)/(272304))

13868.318910337775

## SSIN

In [15]:
result_SSIN=pd.read_csv('test_ret.csv')

## MAE

In [16]:
sum(abs(result_SSIN['pred']-result_SSIN["label"]))/len(result_SSIN)

0.05810492672422826

## RMSE

In [17]:
math.sqrt(sum((result_SSIN['pred']-result_SSIN["label"])**2)/len(result_SSIN))

0.07965967057895923

## SSIN_nor

In [47]:
result_SSIN=pd.read_csv('test_ret.csv')

In [48]:
mean_label = result_SSIN['label'].mean()
std_label = result_SSIN['label'].std()

In [49]:
# 标准化label列
result_SSIN['label_normalized'] = (result_SSIN['label'] - mean_label) / std_label

# 使用相同的平均值和标准差标准化pred列
result_SSIN['pred_normalized'] = (result_SSIN['pred'] - mean_label) / std_label

## MAE

In [50]:
sum(abs(result_SSIN['pred_normalized']-result_SSIN["label_normalized"]))/len(result_SSIN)

2.1707330079326184

## RMSE

In [51]:
math.sqrt(sum((result_SSIN['pred_normalized']-result_SSIN["label_normalized"])**2)/len(result_SSIN))

2.5913567820183205